In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataclasses import dataclass
from pathlib import Path

# import torch as t
from torch.utils.data import DataLoader
from data import NamesDataset
from model import CharLangNet
from dataclasses import dataclass
from lightning.pytorch.loggers import WandbLogger
import wandb
from lightning.pytorch.callbacks import LearningRateMonitor
import lightning as L

In [3]:
DATAROOT = Path.home() / "mldata" / "makemore" 
RUNROOT = Path.home() / "mlruns" / "makemore" / "names"

In [4]:
@dataclass
class Hyperparams:
    context_len: int
    emb_dim: int
    hidden_dim: int
    batch_size: int
    learning_rate: int
    max_epochs: int

In [5]:
hparams = Hyperparams(
    context_len=3,
    emb_dim=2,
    hidden_dim=100,
    batch_size=32,
    learning_rate=0.1,
    max_epochs=100
)

In [6]:
dataset = NamesDataset(DATAROOT / "names.txt", hparams.context_len)
dl = DataLoader(dataset, batch_size=hparams.batch_size, shuffle=True)

In [7]:
lang_net = CharLangNet(
    context_len=hparams.context_len,
    vocab_len=dataset.len_vocab(),
    hidden_dim=hparams.hidden_dim,
    emb_dim=hparams.emb_dim,
    learning_rate=hparams.learning_rate,
    batch_size=hparams.batch_size,
    max_epochs=hparams.max_epochs
)

In [8]:
wandb.finish()
logger = WandbLogger(project="Makemore2Names", save_dir=RUNROOT, log_model="all")
trainer = L.Trainer(
    default_root_dir=RUNROOT,
    max_epochs=hparams.max_epochs,
    logger=logger,
    callbacks=[LearningRateMonitor(logging_interval="epoch")]
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [9]:
trainer.fit(lang_net, dl)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: avilay. Use `wandb login --relogin` to force relogin



  | Name    | Type             | Params
---------------------------------------------
0 | model   | CharLangModel    | 3.5 K 
1 | loss_fn | CrossEntropyLoss | 0     
---------------------------------------------
3.5 K     Trainable params
0         Non-trainable params
3.5 K     Total params
0.014     Total estimated model params size (MB)
/Users/avilayparekh/miniconda3/envs/ai/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [10]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


avg_loss,█▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-SGD,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
avg_loss,2.25618
epoch,99
lr-SGD,0.1
trainer/global_step,712999
